# Trigger SnapMirror Replication within Jupyter Notebook

This notebook demonstrates how to trigger a NetApp SnapMirror update from within a Jupyter Notebook

## Import needed modules

In [1]:
import sys, subprocess

## Install Ansible

In [2]:
%pip install --user ansible netapp-lib

You should consider upgrading via the 'pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


## Set ONTAP cluster/instance connection details

In [3]:
## Enter connection details for the ONTAP cluster/instance on which the destination volume resides
ontapClusterMgmtHostname = '10.61.188.40'
ontapClusterAdminUsername = 'admin'
ontapClusterAdminPassword = 'NetApp!23'
verifySSLCert = 'no' # "yes" or "no"
##

## Set volume details

In [4]:
## Enter source and destination volume details
sourceSvm = 'ailab'
sourceVolume = 'sm' # Note: ONTAP volume name, not Kubernetes PV or PVC name
destinationSvm = 'ai221_data'
destinationVolume = 'sm_dest' # Note: ONTAP volume name, not Kubernetes PV or PVC name
##

## Define Ansible playbook for triggering SnapMirror update

In [5]:
snapMirrorPlaybookContent = '''
---
- name: "Trigger SnapMirror Update"
  hosts: localhost
  tasks:
  - name: update snapmirror
    na_ontap_snapmirror:
      state: present
      source_path: '%s:%s'
      destination_path: '%s:%s'
      hostname: '%s'
      username: '%s'
      password: '%s'
      https: 'yes'
      validate_certs: '%s'
''' % (sourceSvm, sourceVolume, destinationSvm, destinationVolume, ontapClusterMgmtHostname, 
       ontapClusterAdminUsername, ontapClusterAdminPassword, verifySSLCert)
snapMirrorPlaybookFile = open("/home/jovyan/snapmirror-update.yaml", "w")
snapMirrorPlaybookFile.write(snapMirrorPlaybookContent)
snapMirrorPlaybookFile.close()

## Trigger SnapMirror update

In [6]:
result = subprocess.check_output(['/home/jovyan/.local/bin/ansible-playbook', '/home/jovyan/snapmirror-update.yaml'])
print(str(result).replace('\\n', '\n'))

b'
PLAY [Trigger SnapMirror Update] ***********************************************

TASK [Gathering Facts] *********************************************************
ok: [localhost]

TASK [update snapmirror] *******************************************************
ok: [localhost]

PLAY RECAP *********************************************************************
localhost                  : ok=2    changed=0    unreachable=0    failed=0    skipped=0    rescued=0    ignored=0   

'
